In [55]:
! pip install -q --upgrade torch torchvision torchaudio
! pip install -q scikit-learn yfinance pandas matplotlib numpy tensorflow

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
neuralforecast 3.0.2 requires torch<=2.6.0,>=2.0.0, but you have torch 2.8.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import warnings
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import layers, models
warnings.filterwarnings('ignore')

# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.8.0+cpu
CUDA available: False


In [60]:
commodity_tickers = ["ZC=F", "ZS=F", "KE=F"]
stock_tickers = ["DE", "ADM", "CTVA"]

print("Downloading data...")
data_commodities = yf.download(commodity_tickers, start="2020-01-01", end="2024-12-31")['Close']
data_stocks = yf.download(stock_tickers, start="2020-01-01", end="2024-12-31")['Close']

data = pd.concat([data_commodities, data_stocks], axis=1)
data.columns = ['Corn', 'Soybeans', 'Wheat', 'DE', 'ADM', 'CTVA']
data = data.fillna(method='ffill').dropna()

print(f"Data shape: {data.shape}")
data.head()

[*********************100%***********************]  3 of 3 completed
[*********************100%***********************]  3 of 3 completed

Data shape: (1257, 6)


,Corn,Soybeans,Wheat,DE,ADM,CTVA
Date,,,,,,
2020-01-02,485.25,391.50,944.25,39.126690,26.820271,164.021683
2020-01-03,475.00,386.50,930.50,39.050316,26.502977,162.806778
2020-01-06,477.25,384.75,932.75,38.744850,26.465651,163.975266
2020-01-07,475.00,384.50,935.00,38.278133,26.568302,161.128113
2020-01-08,480.00,384.25,938.25,37.853870,26.782942,163.131378


In [61]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.values)

prediction_length = 30
context_length = 90
num_features = data.shape[1]

# ----- Sequence Creation -----
def create_sequences(data, context_length, prediction_length):
    X, y = [], []
    L = len(data)
    for i in range(L - context_length - prediction_length + 1):
        X.append(data[i:i+context_length])
        y.append(data[i+context_length:i+context_length+prediction_length])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

X, y = create_sequences(data_scaled, context_length, prediction_length)

# ----- Train / Val / Test Split -----
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

# ----- TensorFlow Datasets -----
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

In [62]:
model = models.Sequential([
    layers.Input(shape=(context_length, num_features)),
    layers.LSTM(128, return_sequences=True),
    layers.GRU(128),
    layers.Dense(prediction_length * num_features),
    layers.Reshape((prediction_length, num_features))
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")

# ----- Training -----
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=15,
    restore_best_weights=True
)

In [63]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=60,
    callbacks=[early_stopping],
    verbose=1
)

# ----- Evaluation -----
test_loss = model.evaluate(test_dataset)
print("Test Loss:", test_loss)

# ----- Example Forecast -----
y_pred = model.predict(X_test[:1])
y_pred_rescaled = scaler.inverse_transform(y_pred[0])
print("Forecast shape:", y_pred_rescaled.shape)

Epoch 1/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 0.1069 - val_loss: 0.0309
Epoch 2/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0134 - val_loss: 0.0242
Epoch 3/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0102 - val_loss: 0.0276
Epoch 4/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0096 - val_loss: 0.0264
Epoch 5/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0092 - val_loss: 0.0254
Epoch 6/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - loss: 0.0085 - val_loss: 0.0214
Epoch 7/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0080 - val_loss: 0.0274
Epoch 8/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - loss: 0.0074 - val_loss: 0.0221
Epoch 9/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - loss: 0.0063 - val_loss: 0.0210
Epoch 10/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.0060 - val_loss: 0.0173
Epoch 11/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.0054 - val_loss: 0.0200
Epoch 12/60
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - loss: 0.0

In [64]:
y_pred_rescaled

array([[ 573.40295 ,  489.84708 , 1195.3168  ,   53.425842,   45.833378,
         314.95584 ],
       [ 607.0914  ,  484.49164 , 1201.9888  ,   55.382465,   43.940548,
         318.07816 ],
       [ 567.9296  ,  491.34586 , 1198.8961  ,   53.710236,   44.695015,
         323.34552 ],
       [ 604.1428  ,  494.69293 , 1225.5104  ,   54.508118,   44.498817,
         322.5391  ],
       [ 614.5661  ,  494.84076 , 1215.3511  ,   54.72854 ,   45.44645 ,
         331.14584 ],
       [ 596.7669  ,  504.66135 , 1212.0516  ,   55.109585,   43.73358 ,
         322.36047 ],
       [ 603.0707  ,  493.8207  , 1222.3516  ,   54.925613,   46.002483,
         318.0702  ],
       [ 599.5673  ,  502.6597  , 1233.5709  ,   53.01955 ,   45.103653,
         324.30124 ],
       [ 596.2206  ,  496.82346 , 1248.521   ,   53.272144,   44.97423 ,
         320.68436 ],
       [ 595.7841  ,  495.60916 , 1237.8087  ,   54.059628,   44.647762,
         336.5586  ],
       [ 600.20825 ,  486.2368  , 1239.7139  ,   5

In [66]:
import os
import numpy as np
import tensorflow as tf
import joblib 

save_dir = "../models/Commodities_price_Forcasting/"
os.makedirs(save_dir, exist_ok=True)

model_save_path = os.path.join(save_dir, "hybrid_lstm_gru_tf.keras")
scaler_save_path = os.path.join(save_dir, "scaler.pkl")

model.save(model_save_path)
print(f"Model saved at: {model_save_path}")

joblib.dump(scaler, scaler_save_path)
print(f"Scaler saved at: {scaler_save_path}")


def load_forecasting_model(path):
    return tf.keras.models.load_model(path)

def load_scaler(path):
    return joblib.load(path)


def forecast_future(model, recent_data, scaler, context_length, prediction_length, num_features):
    input_seq = recent_data[-context_length:].reshape(1, context_length, num_features)
    pred = model.predict(input_seq, verbose=0)
    pred_rescaled = scaler.inverse_transform(pred[0])
    return pred_rescaled



loaded_model = load_forecasting_model(model_save_path)
loaded_scaler = load_scaler(scaler_save_path)

recent_data = data_scaled[-context_length:]

future_steps = forecast_future(
    loaded_model,
    recent_data,
    loaded_scaler,
    context_length,
    prediction_length,
    num_features
)

print("Forecasted future shape:", future_steps.shape)
print(future_steps[:5])


Model saved at: ../models/Commodities_price_Forcasting/hybrid_lstm_gru_tf.keras
Scaler saved at: ../models/Commodities_price_Forcasting/scaler.pkl
Forecasted future shape: (30, 6)
[[ 468.04602   468.18494  1128.8984     47.939697   44.6193    321.7221  ]
 [ 504.1194    459.30927  1160.5737     50.10257    41.84441   335.4461  ]
 [ 431.7737    467.49683  1157.4352     49.701668   43.45735   338.61383 ]
 [ 511.75314   478.45938  1205.1013     50.13604    41.813538  333.82837 ]
 [ 542.16956   461.09055  1188.1763     50.901012   44.02994   348.09775 ]]
